In [ ]:
import ROOT
# from T2KStyle import T2KStyle
c = ROOT.TCanvas('c', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)

In [ ]:
# 0 - desy
# 1 - mm1
# 2 - er2
# 3 - er3
# 4 - DESY cosmic
# path = '~/LXPLUS/DESY_testbeam/tree/'
path = '/eos/user/s/ssuvorov/DESY_testbeam/cosmic_strict_v2/'
voltage = '380'

file = [ROOT.TFile(path + '../tree/g_' + voltage + '_200_dedx.root')]
file.append(ROOT.TFile(path + '/mm1_' + voltage + 'V_200ns_dedx.root'))
file.append(ROOT.TFile(path + '/er2_' + voltage + 'V_dedx.root'))
file.append(ROOT.TFile(path + '/er3_' + voltage + 'V_dedx.root'))

# file.append(ROOT.TFile('~/LXPLUS/DESY_testbeam/multi_pad/c_0d_0p2T_200ns_50Mhz_dedx.root'))
file.append(ROOT.TFile(path + '../tree/c_0d_0T_200ns_25Mhz_dedx.root'))

name = ['DESY beam', 'MM1', 'ERAM2', 'ERAM3', 'DESY_cosmic', 'DESY_cosmic 2']
color = [ROOT.kBlack, ROOT.kRed, ROOT.kGreen, ROOT.kBlue, ROOT.kViolet, ROOT.kCyan]

## Time difference (dt)

In [ ]:
%jsroot on
cw.Clear()
cw.Divide(2)

hs = [ROOT.THStack('time_1st', ''), ROOT.THStack('time_2nd', '')]
h_name = ['delta_t_fst', 'delta_t_scd']
title = ['t_{2}-t_{1} [time bins]', 't_{3}-t_{1} [time bins]']
var = ['pad_time[1][]-pad_time[0][]', 'pad_time[2][]-pad_time[0][]']
cut = ['multiplicity > 1', 'multiplicity > 2']

for i in [0, 1, 2, 3, 4]:
    for dt_id in range(2):
        dt_h = ROOT.TH1F('dt_h', '', 300, -150., 150)
        file[i].outtree.Project('dt_h', var[dt_id], cut[dt_id], '')
        dt_h.Scale(1./dt_h.Integral())
        dt_h.SetTitle(name[i])
        dt_h.SetLineColor(color[i])
        hs[dt_id].Add(dt_h)
        
for dt_id in range(2):
    cw.cd(dt_id+1)
    hs[dt_id].Draw('histo nostack')
    hs[dt_id].GetXaxis().SetTitle(title[dt_id])
    hs[dt_id].SetMinimum(1e-4)
    hs[dt_id].GetXaxis().SetRangeUser(-60, 80)
    ROOT.gPad.SetLogy()
    ROOT.gPad.SetGrid()
    ROOT.gPad.BuildLegend()

cw.Draw()

In [ ]:
%jsroot on
c.Clear()
c.cd()

h = ROOT.TH2F('h', '', 150, -60., 90., 150, -50., 100)
file[1].outtree.Project('h', 'pad_time[1][]-pad_time[0][]:pad_time[2][]-pad_time[0][]', 'multiplicity > 2', 'colz')
h.GetYaxis().SetTitle('t_{2}-t_{1}')
h.GetXaxis().SetTitle('t_{3}-t_{1}')
h.Draw('colz')
ROOT.gPad.SetGrid()
c.SetLogz(0)
c.SetLogy(0)
c.Draw()

## Charge dependence on dt

In [ ]:
%jsroot on
c.Clear()
c.cd()

file_id = 1

h = ROOT.TH2F('h', '', 300, 0., 3000, 130, -30., 100)
file[file_id].outtree.Project('h', 'pad_time[2][]-pad_time[0][]:pad_charge[0][]', 'multiplicity > 2', 'colz')
h.GetYaxis().SetTitle('t_{3}-t_{1}')
h.GetXaxis().SetTitle('Q_{1}')
h.SetTitle(name[file_id])
ROOT.gStyle.SetOptTitle(1)
h.Draw('colz')
c.SetLogz(0)
c.SetLogy(0)
c.Draw()

In [ ]:
cw.Clear()
cw.cd()
cw.Divide(2)

file_id = 1

h = ROOT.TH2F('h', '', 10, 0., 10, 200, -100., 100)
file[file_id].outtree.Project('h', 'pad_time[1][]-pad_time[0][]:multiplicity', 'multiplicity > 1', 'colz')
h.GetYaxis().SetTitle('t_{2}-t_{1}')
h.GetXaxis().SetTitle('Multiplicity')

h2 = ROOT.TH2F('h2', '', 10, 0., 10, 200, -100., 100)
file[file_id].outtree.Project('h2', 'pad_time[2][]-pad_time[0][]:multiplicity', 'multiplicity > 2', 'colz')
h2.GetYaxis().SetTitle('t_{3}-t_{1}')
h2.GetXaxis().SetTitle('Multiplicity')

cw.cd(1)
h.Draw('colz')
cw.cd(2)
h2.Draw('colz')


cw.Draw()

### Print out fractions with anomlies

In [ ]:
%jsroot on

file_id = 3
h = ROOT.TH1F('h', '', 10, 0., 10.)
fraction_n = file[file_id].outtree.Project('h', 'multiplicity', 'multiplicity > 1 && pad_time[1][]-pad_time[0][] < 0') / file[file_id].outtree.Project('h', '1', 'multiplicity > 1')
print('Fraction dt<0', fraction_n)
fraction_n = file[file_id].outtree.Project('h', 'multiplicity', 'multiplicity > 1 && pad_time[1][]-pad_time[0][] < -2') / file[file_id].outtree.Project('h', '1', 'multiplicity > 1')
print('Fraction dt<-2', fraction_n)
file[file_id].outtree.Project('h', 'multiplicity', 'multiplicity > 1 && pad_time[1][]-pad_time[0][] > 0')
print('av. Mult with dt>0', h.GetMean())
file[file_id].outtree.Project('h', 'multiplicity', 'multiplicity > 1 && pad_time[1][]-pad_time[0][] < -2')
print('av. Mult with dt<-2',  h.GetMean())
print(file[file_id].outtree.Project('h', '1', 'multiplicity > 1 && pad_time[1][]-pad_time[0][] > 0') / file[file_id].outtree.Project('h', '1', 'pad_time[1][]-pad_time[0][] > 0'))
print(file[file_id].outtree.Project('h', '1', 'multiplicity > 1 && pad_time[1][]-pad_time[0][] < 0') / file[file_id].outtree.Project('h', '1', 'pad_time[1][]-pad_time[0][] < 0'))

c.cd()
h = ROOT.TH1F('h', '', 10, 0., 10.)
file[file_id].outtree.Project('h', 'multiplicity', 'multiplicity > 1 && pad_time[1][]-pad_time[0][] > 0')
h.SetTitle('t_{2}-t_{1} > 0')
h2 = ROOT.TH1F('h2', '', 10, 0., 10.)
file[file_id].outtree.Project('h2', 'multiplicity', 'multiplicity > 1 && pad_time[1][]-pad_time[0][] < -2')
h2.SetTitle('t_{2}-t_{1} < -2')

h.Scale(1./h.Integral())
h.Draw('histo')
h2.SetLineColor(ROOT.kRed)
h2.Scale(1./h2.Integral())
h2.Draw('histo same')
c.BuildLegend()
c.Draw()

## Voltage dependence

In [ ]:
%jsroot on
c.Clear()
c.cd()

# ROOT.gStyle.SetOptStat(ROOT.kFALSE)
# f1 = ROOT.TFile(path + '/mm1_360V_200ns_dedx.root')
# f2 = ROOT.TFile(path + '/mm1_380V_200ns_dedx.root')

f1 = ROOT.TFile(path + '/er3_360V_dedx.root')
f2 = ROOT.TFile(path + '/er3_380V_dedx.root')

# f1 = ROOT.TFile(path + '/tree/g_360_200_dedx.root')
# f2 = ROOT.TFile(path + '/tree/g_380_200_dedx.root')

hs = ROOT.THStack('hs', '')

h1 = ROOT.TH1F('h1', '', 200, 0., 200.)
f1.outtree.Project('h1', 'pad_charge[2][]', 'multiplicity > 2 && pad_time[2][]-pad_time[0][] < 2')
# h1.Draw()
h1.SetTitle("360 V")
h1.Scale(1./h1.Integral())
hs.Add(h1)

h2 = ROOT.TH1F('h2', '', 200, 0., 200)
f2.outtree.Project('h2', 'pad_charge[2][]', 'multiplicity > 2 && pad_time[2][]-pad_time[0][] < 2', '')
h2.Scale(1./h2.Integral())
h2.SetLineColor(ROOT.kRed)
h2.SetTitle("380 V")
hs.Add(h2)

hs.Draw('histo nostack')
hs.GetXaxis().SetTitle('Q_{3}')
c.BuildLegend()
c.Draw()

In [ ]:
%jsroot on
c.Clear()
c.cd()

hs = ROOT.THStack('hs', '')

h1 = ROOT.TH1F('h1', '', 100, 0., 3000.)
file[1].outtree.Project('h1', 'pad_charge[2][]', 'multiplicity > 2 && pad_time[2][]-pad_time[0][] < 2')
h1.SetTitle('360 V')
h1.Scale(1./h1.Integral())
hs.Add(h1)

h2 = ROOT.TH1F('h2', '', 100, 0., 3000)
file[1].outtree.Project('h2', 'pad_charge[2][]', 'multiplicity > 2 && pad_time[2][]-pad_time[0][] < 2', '')
h2.Scale(1./h2.Integral())
h2.SetTitle('t_{3}-t_{1} < 2')
h2.SetLineColor(ROOT.kRed)
hs.Add(h2)

hs.Draw('histo nostack')
hs.GetXaxis().SetTitle('Q_{3}')
c.BuildLegend()
c.Draw()

## Charge fraction

In [ ]:
file_new = [ROOT.TFile(path + '../tree/g_360_200_dedx.root')]
file_new.append(ROOT.TFile(path + '../tree/mm1_360V_200ns_dedx.root'))
file_new.append(ROOT.TFile(path + '../tree/er2_360V_dedx.root'))
file_new.append(ROOT.TFile(path + '../tree/er3_360V_dedx.root'))

# file.append(ROOT.TFile('~/LXPLUS/DESY_testbeam/multi_pad/c_0d_0p2T_200ns_50Mhz_dedx.root'))
file_new.append(ROOT.TFile('~/LXPLUS/DESY_testbeam/tree/c_0d_0T_200ns_25Mhz_dedx.root'))

name = ['DESY beam', 'MM1', 'ERAM2', 'ERAM3', 'DESY_cosmic', 'DESY_cosmic 2']
color = [ROOT.kBlack, ROOT.kRed, ROOT.kGreen, ROOT.kBlue, ROOT.kViolet, ROOT.kCyan]

hs = ROOT.THStack('hs', '')
for i in [0, 1, 2, 3]:
    h2 = ROOT.TH1F('h2', '', 300, 0., .3)
    file_new[i].outtree.Project('h2', 'pad_charge[2][]/pad_charge[0][]', 'multiplicity > 2 && pad_time[2][]-pad_time[0][] < 2', '')
    h2.Scale(1./h2.Integral())
    h2.SetLineColor(color[i])
    h2.SetTitle(name[i])
    hs.Add(h2)
    
c.cd()
hs.Draw('histo nostack')
hs.GetXaxis().SetTitle('Q_{3}/Q_{1}')
c.Draw()
c.BuildLegend()

## Position dependence

In [ ]:
%jsroot on
c.Clear()
c.cd()

ROOT.gStyle.SetOptStat(ROOT.kFALSE)
file_id = 1

h = ROOT.TH2F('h', '', 36, 0., 36., 130, -30., 100)
file[file_id].outtree.Project('h', 'pad_time[2][]-pad_time[0][]:pad_y[2]', 'multiplicity > 2', 'colz')
h.GetYaxis().SetTitle('t_{3}-t_{1}')
h.GetXaxis().SetTitle('3rd pad Y')
h.SetTitle(name[file_id])
ROOT.gStyle.SetOptTitle(1)
h.Draw('colz')
c.Draw()

In [ ]:
%jsroot on
c.Clear()
c.cd()

ROOT.gStyle.SetOptStat(ROOT.kFALSE)
file_id = 1

h = ROOT.TH2F('h', '', 36, 0., 36., 130, -30., 100)
file[file_id].outtree.Project('h', 'pad_time[2][]-pad_time[0][]:pad_x[2]', 'multiplicity > 2', 'colz')
h.GetYaxis().SetTitle('t_{3}-t_{1}')
h.GetXaxis().SetTitle('3rd pad X')
h.SetTitle(name[file_id])
ROOT.gStyle.SetOptTitle(1)
h.Draw('colz')
c.Draw()

In [ ]:
%jsroot on
c.Clear()
c.cd()

ROOT.gStyle.SetOptStat(ROOT.kFALSE)
file_id = 1

h = ROOT.TH2F('h', '', 10, -5., 5., 130, -30., 100.)
file[file_id].outtree.Project('h', 'pad_time[2][]-pad_time[0][]:pad_y[2][] - pad_y[0][]', 'multiplicity > 2', 'colz')
h.GetYaxis().SetTitle('t_{3}-t_{1}')
h.GetXaxis().SetTitle('x_{3} - x_{0}, [pads]')
h.SetTitle(name[file_id])
ROOT.gStyle.SetOptTitle(1)
h.Draw('colz')
c.Draw()

In [ ]:
%jsroot on
c.Clear()
c.cd()

ROOT.gStyle.SetOptStat(ROOT.kFALSE)
file_id = 1

h = ROOT.TH2F('h', '', 37, 0., 37., 37, 0., 37)
file[file_id].outtree.Project('h', 'pad_y[2]:pad_x[2]', 'multiplicity > 2 && pad_time[2][] - pad_time[0][] < 2', 'colz')
h_norm = ROOT.TH2F('hn', '', 37, 0., 37., 37, 0., 37)
file[file_id].outtree.Project('hn', 'pad_y[2]:pad_x[2]', 'multiplicity > 2', 'colz')
h.Divide(h_norm)
h.GetYaxis().SetTitle('Y')
h.GetXaxis().SetTitle('X')
h.SetTitle(name[file_id])
ROOT.gStyle.SetOptTitle(1)
h.Draw('colz')
c.Draw()

## Angle dependence

In [ ]:
c.Clear()
c.cd()
file_id = 4
h = ROOT.TH2F('h', '', 300, -150., 150., 30, 0., 0.3)
file[file_id].outtree.Project('h', 'angle_yz:pad_time[2][]-pad_time[0][]')
h.GetYaxis().SetRangeUser(0., 0.4)
h.SetTitle(name[file_id])
h.GetYaxis().SetTitle('tan(#theta)')
h.GetXaxis().SetTitle('t_{3} - t_{1}')
h.Draw('colz')
c.SetLogz()
c.SetLogy(0)
c.Draw()

In [ ]:
%jsroot on
c.Clear()
c.cd()
file_id = 3
h = ROOT.TH2F('h', '', 300, -150., 150., 30, 0., 0.3)
file[file_id].outtree.Project('h', 'angle_xy:pad_time[2][]-pad_time[0][]')
h.GetYaxis().SetRangeUser(0., 0.4)
h.SetTitle(name[file_id])
h.GetYaxis().SetTitle('tan(#phi)')
h.GetXaxis().SetTitle('t_{3} - t_{1}')
h.Draw('colz')
c.SetLogz()
c.SetLogy(0)
c.Draw()